# Sensor Position Optimization

### BMW Quantum Computing Challenge

#### Valter Uotila, Unified Database Management Systems, Department of Computer Science at University of Helsinki
#### Sardana Ivanova, Discovery Research Group, Department of Computer Science at University of Helsinki

## Remarks about the implementation

We have been learning, testing and developing some basic quantum algorithms using D-waves own API but we are not very experienced with details related to these quantum computers.

Unfortunately, it seems that Amazon Bracket does not yet support D-waves hybrid solvers. We might be wrong but we could not figure out how to access, for example, LeapHybridSampler using Bracket. It seems that BracketDWaveSampler and BracketSampler are using the standard DWaveSampler (https://docs.ocean.dwavesys.com/projects/system/en/stable/reference/samplers.html). Because our solution is based on LeapHybridSampler, it is not possible to run it in Bracket. Maybe in the future Amazon will implement something like BracketDWaveHybridSampler which offers the functionality of LeapHybridSampler.

Anyway, Bracket allows some hybrid solving properties using PennyLane but we are not familiar with them yet. We are also positively impressed how Bracket collects various quantum computing resources together.

## Initializing parameters

### Importing D-wave packages

Note that in order to run the codes here, you need to be able to successfully access your D-wave quantum cloud computing resources. You can see more info at https://cloud.dwavesys.com/. We are used to develop with D-waves developer's plan which includes 1 min of quantum computing time per month.

In [1]:
import dimod

from dwave.system import LeapHybridSampler

import json
import itertools
import os
import math

from ipynb.fs.defs.sensor_covers_2D import sensor_covers

notebook_path = os.path.abspath("main_2D_connecting_Dwave_Leap.ipynb")

### Initializing sensors

In [2]:
abs_sensors_file_path = os.path.join(os.path.dirname(notebook_path), "2d_example_data/sensors.json")

f = open(abs_sensors_file_path)

sensor_root = json.load(f)
sensors = sensor_root["sensors"]

# print(sensors)

### Initializing car

In [3]:
abs_car_file_path = os.path.join(os.path.dirname(notebook_path), "2d_example_data/2D_allowed_sensors_positions.json")

f = open(abs_car_file_path)

car = json.load(f)

car_grid = [(0,3), (1,3), (2,3), (0,2), (1,2), (2,2), (0,1), (1,1), (2,1), (0,0), (1,0), (2,0)]

# print(car)

### Initializing environment

In [4]:
environment = {"v1": (-8,8), "v2": (8,8), "v3": (8, -8), "v4": (-8,-8)}

def critical_value(x, y):
    return 1

## Constructing quadratic binary model

In [5]:
bqm = dimod.BinaryQuadraticModel({}, {}, 0.0, dimod.BINARY)

# Initializing variables
# Variables are triples (x, y, i) where x and y are coordinate points in the space
# and i is the index of the sensor. For each sensor exactly two variables are two
# and another one of them is on the car.

def create_variables(start_x, end_x, start_y, end_y, sensors):
    variables = []
    for x in range(start_x, end_x + 1):
        for y in range(start_y, end_y + 1):
            if (x,y) not in car_grid:
                variables.append((x, y))
    return variables

env = create_variables(environment["v1"][0], environment["v2"][0], environment["v3"][1], environment["v2"][1], sensors)

# print(len(env))

# Next we collect the variables which are assigned to the car surface

car_surface = []

for pos in car["positions"]:
    if pos["x1"] == pos["x2"]:
        x = pos["x1"]
        for y in range(pos["y1"], pos["y2"] + 1):
            car_surface.append((x, y))
    elif pos["y1"] == pos["y2"]:
        y = pos["y1"]
        for x in range(pos["x1"], pos["x2"] + 1):
            car_surface.append((x, y))
    else:
        raise ValueError('The allowed position on the car surface is not a line.')
        
# print(car_surface)

In [6]:
def append_linear_safe(x, value):
    try:
        bqm.set_linear(x, bqm.get_linear(x) + value)
    except:
        bqm.set_linear(x, value)

def append_quadratic_safe(x, value):
    try:
        bqm.quadratic[x] = bqm.quadratic[x] + value
    except:
        bqm.quadratic[x] = value
        
def distance(x, y):
    return math.floor(math.sqrt(pow((x[0]- y[0]), 2) + pow((x[0]- y[0]),2)))

### Constraint 1: variables go in pairs

In [7]:
# Encoding constraint H1

c = 150

for sensor in sensors:
    for y in env:
        var_y = (y[0], y[1], sensor["id"])
        append_linear_safe(var_y, c)
        for x in car_surface:
            var_x = (x[0], x[1], sensor["id"])
            append_linear_safe(var_x, c)
            append_quadratic_safe((var_x, var_y), -2*c)
            
        for tupl in itertools.combinations(car_surface, 2):
            var_1 = (tupl[0][0], tupl[0][1], sensor["id"])
            var_2 = (tupl[1][0], tupl[1][1], sensor["id"])
            append_quadratic_safe((var_1, var_2), 2*c)
            
            
# Encoding constraint H2

for sensor in sensors:
    for x in car_surface:
        var_x = (x[0], x[1], sensor["id"])
        append_linear_safe(var_x, c)
        for y in env:
            var_y = (y[0], y[1], sensor["id"])
            append_linear_safe(var_y, c)
            append_quadratic_safe((var_x, var_y), -2*c)
            
        for tupl in itertools.combinations(env, 2):
            var_1 = (tupl[0][0], tupl[0][1], sensor["id"])
            var_2 = (tupl[1][0], tupl[1][1], sensor["id"])
            append_quadratic_safe((var_1, var_2), 2*c)
            
#for elem in bqm.linear:
#        print(elem, bqm.linear[elem])

#for elem in bqm.quadratic:
#    print(elem, bqm.quadratic[elem])
            

### Constraint 2: range of sensor views

In [8]:
# Encoding constraint H3

h = 1

for sensor in sensors:
    for x in car_surface:
        for y in env:
            constant = pow(distance(x, y) - sensor["view"]["range"] + 1, 2)
            #if constant != 1:
            #    constant = pow(distance(x, y) + 1 - sensor["view"]["range"] + 1, 2)
            var_x = (x[0], x[1], sensor["id"])
            append_linear_safe(var_x, h*constant)
            
            var_y = (y[0], y[1], sensor["id"])
            append_linear_safe(var_y, h)
            
            append_quadratic_safe((var_x, var_y), -2*constant*h)
    

### Constraint 3: maximizing sensor view coverage

Other constaints try to minimize the number of sensors. Thus the trivial solution to these is not to include any sensors. On the other hand, this part of the model tries to maximize the coverage. This creates a sort of tension between different parts of this optimization problem which eventually unstabilizes the energy and creates some non-trivial solution to the problem.

In [9]:
# Encoding constraint H4

total_points = len(env)

d = 1

for sensor in sensors:
    for x in car_surface:
        for y in env:
            covered_area = pow(sensor["view"]["range"], 2)*math.tan(math.pi * sensor["view"]["angle"] / 360)
            var_x = (x[0], x[1], sensor["id"])
            var_y = (y[0], y[1], sensor["id"])
            append_quadratic_safe((var_x, var_y), d*(pow(covered_area, 2) - 2*covered_area*total_points))
            

### Constraint 4: optimizing overlap of sensor views

In [10]:
# Encoding constraint H5

for p in env:
    cr = critical_value(p[0], p[1])
    for sensor in sensors:
        for x in car_surface:
            for y in env:
                covers = sensor_covers(x, y, p, sensor)
                
                var_x = (x[0], x[1], sensor["id"])
                var_y = (y[0], y[1], sensor["id"])
            
                append_quadratic_safe((var_x, var_y), covers - 2*cr*covers)

### Constraint 5: minimizing total price

In [11]:
# Encoding constraint H6

for sensor in sensors:
    for x in car_surface:
        for y in env:
            var_x = (x[0], x[1], sensor["id"])
            var_y = (y[0], y[1], sensor["id"])
            
            append_quadratic_safe((var_x, var_y), sensor["price"])


### Solve QUBO

In [12]:
sampler = LeapHybridSampler()
sampleset = sampler.sample(bqm, label='BMW Sensor Position')
sample = sampleset.first.sample
# energy = sampleset.first.energy
print("Possible sensor positions in the space (x-coordinate, y-coordinate, sensor id):")
for varname, value in sample.items():
    if value == 1:
        print(varname, value)

Possible sensor positions in the space (x-coordinate, y-coordinate, sensor id):
